# CAMEL Cookbook - Object Detection with ACI.dev MCP Tools

**Description:** Learn how to build an object detection agent using CAMEL AI and ACI.dev's MCP protocol for seamless ML tasks. 

⭐ Star us on [GitHub](https://github.com/camel-ai/camel), join our [Discord](https://discord.gg/EXAMPLE), or follow us on [X](https://x.com/camelaiorg)

This cookbook shows how to build a powerful object detection agent using CAMEL AI connected to ACI.dev's MCP tools. We'll create an agent that analyzes images, detects objects like cars or trees, and explains results in natural language—all without writing complex ML code.

**Key Learnings:**
- Why agents need tools to be truly useful.
- How MCP enables dynamic, aware tool usage for tasks like object detection.
- Setting up CAMEL with ACI.dev for real-time image analysis.
- Building and running your own object detection agent.
- Handling outputs with summaries, tables, and visualized results.

This setup uses CAMEL's `MCPToolkit` to connect to ACI.dev's MCP servers, powering object detection via Replicate's ML models.

### 📦 Installation

In [ ]:
%pip install camel-ai aci-mcp aci-sdk

Set up keys (ACI_API_KEY, LINKED_ACCOUNT_OWNER_ID, GOOGLE_API_KEY, REPLICATE_API_TOKEN) in `.env` file.

In [1]:
import os
from dotenv import load_dotenv

### Define CAMEL agents

In [2]:
agent_name = "ObjectDetectionAgent"
system_prompt="""
You are a specialized Object Detection Agent. Your primary function is to use the `REPLICATE.run` tool for object detection and present the findings in a user-friendly format. "
"The user will provide a text prompt containing an image URL and a query. You must extract the `image` URL and the `query` object(s). "
"Immediately call the `REPLICATE.run` tool. The `input` must be a dictionary with two keys: `image` (the URL) and `query` (a string of the object(s)). "
"Do not ask for clarification; make a reasonable inference if the query is ambiguous. "
"After receiving the tool's output, format your response as follows: "
"- **Natural Language Summary:** Start with a detailed friendly, insightful analysis of the detection results in plain English. "
"- **Markdown Table:** Create a markdown table with columns: 'Object', 'Confidence Score', and 'Bounding Box Coordinates'. "
"- **Result Image:** If the tool provides a URL for an image with bounding boxes, display it using markdown: `![Detected Objects](URL_HERE)`. "
"Whenever I give you a link, trigger the tool call, extract its outputs and links, and present me in a proper markdown format with detailed analysis from the tool call in natural language.
"""

### MCP servers configuration using ACI.dev

In [3]:
from camel.toolkits import MCPToolkit
mcp_config = {
    "mcpServers": {
        "aci_apps": {
            "command": "aci-mcp",
            "args": [
                "apps-server",
                "--apps=REPLICATE",
                "--linked-account-owner-id",
                "parthshr370"
            ],
            "env": {"ACI_API_KEY": os.getenv("ACI_API_KEY")},
        }
    }
}
mcp_toolkit = MCPToolkit(config_dict=mcp_config)
await mcp_toolkit.connect()


Define the CAMEL agent with GEMINI 2.5 Flash model with Replicate MCP tools!

In [4]:
from camel.agents import ChatAgent
from camel.messages import BaseMessage
from camel.models import ModelFactory
from camel.types import ModelPlatformType

tools = mcp_toolkit.get_tools()

# Initialize Gemini model
model = ModelFactory.create(
    model_platform=ModelPlatformType.GEMINI,
    model_type="gemini-2.5-flash",
    api_key=os.getenv("GOOGLE_API_KEY"),
    model_config_dict={"temperature": 0.0, "max_tokens": 4096},
)

# Create system message
sys_msg = BaseMessage.make_assistant_message(
    role_name=agent_name,
    content=system_prompt,
)

agent = ChatAgent(model=model, system_message=sys_msg, tools=tools, memory=None)

After create the agent, let's do some examples using Replicate to identify the following images:

![](https://images.pexels.com/photos/2255935/pexels-photo-2255935.jpeg)
![](https://www.livemint.com/rf/Image-621x414/LiveMint/Period1/2012/10/01/Photos/Road621.jpg)
![](https://media.business-humanrights.org/media/images/16278498935_dac4d8f223_o.2e16d0ba.fill-1000x1000-c50.jpg)

In [5]:
tasks = (
    "Analyze the vegetable stall and identify all produce, including tomato, onion, cabbage, cucumber, zucchini, carrot, and beet, in this image: https://images.pexels.com/photos/2255935/pexels-photo-2255935.jpeg",
    "Analyze the busy street scene and identify all vehicles, such as car, bus, and truck, as well as people, in this image: https://www.livemint.com/rf/Image-621x414/LiveMint/Period1/2012/10/01/Photos/Road621.jpg",
    "Analyze the warehouse scene and identify persons, cardboard boxes, and conveyor belts in this image: https://media.business-humanrights.org/media/images/16278498935_dac4d8f223_o.2e16d0ba.fill-1000x1000-c50.jpg"
)

for task in tasks:
    response = await agent.astep(task)
    print(response.msg.content)

Here's an analysis of the detected produce in the vegetable stall image:

**Natural Language Summary:**
The object detection model successfully identified several types of produce from your query. We found multiple instances of **onions**, **cabbage**, **tomatoes**, and **carrots** with varying confidence levels. A single **zucchini** and a **cucumber** were also detected. Interestingly, a **beet** was identified, though with a lower confidence score. The model seems to have accurately pinpointed the locations of these vegetables within the image, providing bounding box coordinates for each detection.

**Detection Results:**

| Object | Confidence Score | Bounding Box Coordinates (y_min, x_min, y_max, x_max) |
|---|---|---|
| carrot | 0.465 | [294, 916, 1001, 1438] |
| cabbage | 0.497 | [110, 1982, 1436, 3150] |
| onion | 0.412 | [742, 3535, 1378, 4186] |
| onion | 0.423 | [778, 4032, 1507, 4837] |
| onion | 0.373 | [10, 3733, 785, 4501] |
| cabbage | 0.328 | [6, 3088, 1022, 3850] |
| 

Finally, we disconnect the MCP toolkit. 

In [6]:
await mcp_toolkit.disconnect()